## Offline Data Processing

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import data_process as dp
import json

In [ ]:
# run a single update of the data pull
with open('../auths.json', 'r') as fp:
    api_args = json.load(fp)

time_args = {
    'start_year': 2018,'start_month': 8,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2018,'end_month': 8,'end_day': 1,'end_hour': 0,'end_minute': 30,'end_second': 0,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data.csv'
}
api_args.update(time_args)

dp.pull_offline_data(**api_args)

In [2]:
# download data in a loop
time_args = [
    {
    'start_year': 2018,'start_month': 7,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2018,'end_month': 12,'end_day': 31,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jul2dec2018.csv'
    },
    {
    'start_year': 2019,'start_month': 1,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2019,'end_month': 6,'end_day': 30,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jan2jun2019.csv'  
    },
    {
    'start_year': 2019,'start_month': 7,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2019,'end_month': 12,'end_day': 31,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jul2dec2019.csv'  
    },
    {
    'start_year': 2020,'start_month': 1,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2020,'end_month': 6,'end_day': 15,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jan2jun2020.csv'  
    }
]
for i in time_args:
    with open('../auths.json', 'r') as fp:
        api_args = json.load(fp)
    api_args.update(i)
    dp.pull_offline_data(**api_args)
    print("Success!")

Success!
Success!
Success!
Success!


## Create meta_data

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
import pandas as pd
import json
import numpy as np

### Create column_aliases

In [2]:
df = pd.read_csv('../data/raw_data/alumni_data_jul2dec2018.csv', index_col='time')

d1 = {'column_names': list(df.columns)}
column_aliases = [
    'pchwst', 'vrf50', 'oat', 'sat', 'sat_stpt', 'oah', 'vrf67', 'pchw_flow',
    'hwe', 'vrf1', 'vrf30', 'vrf34', 'vrf74', 'cwe', 'hws_st_stpt', 'vrf60',
    'vrf63', 'hws_st', 'hws_vlv1', 'vrf77', 'vrf64', 'vrf10', 'ee', 'hws_rt',
    'vrf100', 'vrf40', 'hws_flow', 'vrf108', 'vrf20'
]

d2 = {}
for i, j in zip(df.columns, column_aliases):
    d2.update({j:i})
d1['column_aliases'] = d2

### Create column stats

In [3]:
stats = {}
d3 = dict(df.describe())
for key in d3.keys():
    stats[key] = dict(d3[key])
d1['column_stats'] = stats

### Dump meta data

In [4]:
with open('../alumni_scripts/meta_data.json', 'w') as fp:
    json.dump(d1, fp, indent=4)

### Read meta data

In [5]:
with open('../alumni_scripts/meta_data.json', 'r') as fp:
        meta_data_ = json.load(fp)
meta_data = meta_data_.copy()
for key, value in meta_data_['column_stats'].items():
    if value['std'] == 0:
        meta_data['column_stats'][key]['std'] = 0.0001  # add small std for constant values
stats = pd.DataFrame(meta_data['column_stats'])